# Crypto Currency Exploration
# Data from RapidAPI
## Contact info:
### Sean Pharris - pharris.sean@gmail.com

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crypto-files/Crypto-proj/Ontology.csv
/kaggle/input/crypto-files/Crypto-proj/Binance-USD.csv
/kaggle/input/crypto-files/Crypto-proj/Monero.csv
/kaggle/input/crypto-files/Crypto-proj/Storm.csv
/kaggle/input/crypto-files/Crypto-proj/Enjin-Coin.csv
/kaggle/input/crypto-files/Crypto-proj/Qtum.csv
/kaggle/input/crypto-files/Crypto-proj/Civic.csv
/kaggle/input/crypto-files/Crypto-proj/Compound.csv
/kaggle/input/crypto-files/Crypto-proj/DATAcoin.csv
/kaggle/input/crypto-files/Crypto-proj/Zcash.csv
/kaggle/input/crypto-files/Crypto-proj/ICON.csv
/kaggle/input/crypto-files/Crypto-proj/Tezos.csv
/kaggle/input/crypto-files/Crypto-proj/Komodo.csv
/kaggle/input/crypto-files/Crypto-proj/Storj.csv
/kaggle/input/crypto-files/Crypto-proj/aelf.csv
/kaggle/input/crypto-files/Crypto-proj/TerraUSD.csv
/kaggle/input/crypto-files/Crypto-proj/BitTorrent.csv
/kaggle/input/crypto-files/Crypto-proj/Waves.csv
/kaggle/input/crypto-files/Crypto-proj/ZenCash.csv
/kaggle/input/crypto-files/Crypto-proj/C

In [2]:
pip install alpha_vantage

Note: you may need to restart the kernel to use updated packages.


API used for Crypto data is from RapidAPI - Alpha Vantage. https://rapidapi.com/alphavantage/api/alpha-vantage/   
Documentation - https://www.alphavantage.co/documentation/  
Repo for API code - https://github.com/RomelTorres/alpha_vantage  
Code doc - https://alpha-vantage.readthedocs.io/en/latest/


In [3]:
# import digital currency list (Contains all Crypto names and codes)
df = pd.read_csv('/kaggle/input/curr-list/currency_list.csv')

In [4]:
# df_dict is a dictionary that contatins all currency dataframes
df_dict = {'name':[]}

for name in df['currency name']:
    df_dict['name'].append(name)
    
for name in df['currency name']:
    csv_path = "/kaggle/input/crypto-files/Crypto-proj/" + name + ".csv"
    df_dict[name] = pd.read_csv(csv_path)

In [5]:
df_dict['Aave']

,date,1a. open (CNY),1b. open (USD),2a. high (CNY),2b. high (USD),3a. low (CNY),3b. low (USD),4a. close (CNY),4b. close (USD),5. volume,6. market cap (USD)
0,2022-02-09,1171.483530,184.100,1175.937840,184.800,1121.849790,176.300,1135.212720,178.400,24410.437,24410.437
1,2022-02-08,1160.029590,182.300,1226.207910,192.700,1104.032550,173.500,1170.210870,183.900,154091.746,154091.746
2,2022-02-07,1104.668880,173.600,1172.119860,184.200,1080.488340,169.800,1158.756930,182.100,119127.010,119127.010
3,2022-02-06,1057.580460,166.200,1110.395850,174.500,1051.853490,165.300,1104.032550,173.500,91113.374,91113.374
4,2022-02-05,1061.398440,166.800,1104.032550,173.500,1044.217530,164.100,1057.580460,166.200,100784.207,100784.207
...,...,...,...,...,...,...,...,...,...,...,...
478,2020-10-19,259.749906,40.820,260.895300,41.000,228.518830,35.912,228.951534,35.980,86475.936,86475.936
479,2020-10-18,263.179725,41.359,263.599702,41.425,250.892192,39.428,259.304475,40.750,30917.953,30917.953
480,2020-10-17,250.529484,39.371,275.117276,43.235,243.116240,38.206,263.217904,41.365,41971.143,41971.143
481,2020-10-16,273.730076,43.017,278.712540,43.800,248.754124,39.092,251.102181,39.461,90836.412,90836.412


### The loop below calls the api gathers todays currency records and appends them to the datafame within the dictionary and the CSV. Alpha Vantage keeps records for 1000 days I am appending to the current data instead of capturing the data passed through the API so that each currencies data can exceed 1000 days.

In [6]:
'''import requests
import time
import json
from csv import DictWriter
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from datetime import date

today = date.today()
date = today.strftime("%Y-%m-%d")

headersCSV = ['date','1a. open (CNY)', '1b. open (USD)', '2a. high (CNY)', '2b. high (USD)',
       '3a. low (CNY)', '3b. low (USD)', '4a. close (CNY)', '4b. close (USD)',
       '5. volume', '6. market cap (USD)']

for n, code in enumerate(df['currency code']):
       
    time.sleep(20)
    
    data = ""
    working_currency = df['currency name'][n]
    currency_csv = working_currency + ".csv"
    csv_path = "./" + currency_csv
    
    try:
        
        cc = CryptoCurrencies(key= API_KEY, rapidapi=True, output_format='pandas')
        
        data, meta_data = cc.get_digital_currency_daily(symbol=code, market='CNY')
    
        new_data = data.loc[date].to_numpy()
    
        open_cny = new_data[0][0]
        open_usd = new_data[0][1]
        high_cny = new_data[0][2]
        high_usd = new_data[0][3]
        low_cny = new_data[0][4]
        low_usd = new_data[0][5]
        close_cny = new_data[0][6]
        close_usd = new_data[0][7]
        volume = new_data[0][8]
        market_cap = new_data[0][9]
    
        df_dict[working_currency].to_csv(currency_csv, index=False)

        dict={'date':date, '1a. open (CNY)': open_cny, '1b. open (USD)': open_usd, '2a. high (CNY)': high_cny, '2b. high (USD)': high_usd,
           '3a. low (CNY)': low_cny, '3b. low (USD)': low_usd, '4a. close (CNY)': close_cny, '4b. close (USD)': close_usd,
           '5. volume': volume, '6. market cap (USD)': market_cap}

        with open(csv_path, 'a', newline='') as f_object:  
    
            dictwriter_object = DictWriter(f_object, fieldnames=headersCSV)
    
            dictwriter_object.writerow(dict)
    
            f_object.close()
            
        df_dict[working_currency] = pd.read_csv(csv_path)

        df_dict[working_currency].sort_values(by='date', inplace=True)
        
    except(IndexError, ValueError):
        print("Could not retrieve data for ", working_currency)
        pass'''

'import requests\nimport time\nimport json\nfrom csv import DictWriter\nfrom alpha_vantage.cryptocurrencies import CryptoCurrencies\nfrom datetime import date\n\ntoday = date.today()\ndate = today.strftime("%Y-%m-%d")\n\nheadersCSV = [\'date\',\'1a. open (CNY)\', \'1b. open (USD)\', \'2a. high (CNY)\', \'2b. high (USD)\',\n       \'3a. low (CNY)\', \'3b. low (USD)\', \'4a. close (CNY)\', \'4b. close (USD)\',\n       \'5. volume\', \'6. market cap (USD)\']\n\nfor n, code in enumerate(df[\'currency code\']):\n       \n    time.sleep(20)\n    \n    data = ""\n    working_currency = df[\'currency name\'][n]\n    currency_csv = working_currency + ".csv"\n    csv_path = "./" + currency_csv\n    \n    try:\n        \n        cc = CryptoCurrencies(key= API_KEY, rapidapi=True, output_format=\'pandas\')\n        \n        data, meta_data = cc.get_digital_currency_daily(symbol=code, market=\'CNY\')\n    \n        new_data = data.loc[date].to_numpy()\n    \n        open_cny = new_data[0][0]\n     

In [7]:
# clears csv

'''df_dict['Aave'] = open('Aave.csv', 'r+')
df_dict['Aave']. truncate(0)
df_dict['Aave']. close()'''

"df_dict['Aave'] = open('Aave.csv', 'r+')\ndf_dict['Aave']. truncate(0)\ndf_dict['Aave']. close()"